# Convolutional Neural Network

### Importing the libraries

In [24]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [25]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [26]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 7991 images belonging to 2 classes.


### Preprocessing the Test set

In [27]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1999 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [28]:
cnnt = tf.keras.models.Sequential()

### Step 1 - Convolution

In [29]:
cnnt.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

### Step 2 - Pooling

In [30]:
cnnt.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [31]:
cnnt.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnnt.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a third convolutional layer

In [32]:
cnnt.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnnt.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Step 3 - Flattening

In [33]:
cnnt.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [34]:
cnnt.add(tf.keras.layers.Dense(units=128,activation='relu'))

### Step 5 - Output Layer

In [35]:
cnnt.add(tf.keras.layers.Dense(units=2,activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [36]:
cnnt.compile(optimizer='nadam',loss='categorical_crossentropy',metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [37]:
cnnt.fit(x = training_set,validation_data = test_set,epochs= 25)

Epoch 1/25


2023-09-22 01:24:06.471412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - ETA: 0s - loss: 0.6751 - accuracy: 0.5743

2023-09-22 01:24:21.171891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - 17s 67ms/step - loss: 0.6751 - accuracy: 0.5743 - val_loss: 0.6356 - val_accuracy: 0.6603
Epoch 2/25
250/250 [==============================] - 16s 65ms/step - loss: 0.6017 - accuracy: 0.6770 - val_loss: 0.5607 - val_accuracy: 0.7139
Epoch 3/25
250/250 [==============================] - 16s 66ms/step - loss: 0.5494 - accuracy: 0.7172 - val_loss: 0.4985 - val_accuracy: 0.7574
Epoch 4/25
250/250 [==============================] - 16s 63ms/step - loss: 0.4998 - accuracy: 0.7506 - val_loss: 0.4786 - val_accuracy: 0.7699
Epoch 5/25
250/250 [==============================] - 16s 63ms/step - loss: 0.4764 - accuracy: 0.7715 - val_loss: 0.4843 - val_accuracy: 0.7689
Epoch 6/25
250/250 [==============================] - 17s 68ms/step - loss: 0.4495 - accuracy: 0.7850 - val_loss: 0.4200 - val_accuracy: 0.8099
Epoch 7/25
250/250 [==============================] - 16s 66ms/step - loss: 0.4284 - accuracy: 0.7986 - val_loss: 0.4319 - val_accuracy: 0.8024
Epo

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.utils import load_img
from keras.utils import img_to_array

# Load the test image from the file path and resize it to the target size (64x64 pixels)
test_image = load_img('/Users/surya/Documents/Surya/Git/Repo1/DL practice/Convolutional Neural Networks/dataset/test_set/cats/cat.4750.jpg', target_size=(64, 64))

# Convert the test image to a NumPy array
test_image = img_to_array(test_image)/255.

# Add an extra dimension to the test image array to match the input shape expected by the model
test_image = np.expand_dims(test_image, axis=0)

# Use the CNN model to make a prediction on the test image
result = cnnt.predict(test_image)

# Get the class indices mapping from the training set (used for interpreting the predictions)
class_indices = training_set.class_indices

# Check the prediction result and assign the corresponding label (dog or cat)
if result[0][0] > 0 and result[0][0] < 0.5:
    prediction = 'dog'
else:
    prediction = 'cat' 

# Print the final prediction (dog or cat)
print(prediction)
print(result)


1/1 [==============================] - 0s 61ms/step
cat
[[0.6904061  0.30959392]]


In [ ]:
a = np.array(cnnt.weights)
print(a.shape)

(10,)


/var/folders/21/yhy2dhmx219b7cbcq243hmwc0000gn/T/ipykernel_2138/3973995084.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(cnnt.weights)


In [ ]:
# Save the model's architecture to a JSON file
model_json = cnnt.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model's weights to an HDF5 file
cnnt.save_weights("model_weights.h5")